In [72]:
import requests
import os
import pandas as pd
import pickle
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

### 크롤링

In [ ]:
try:
    os.makedirs('./tmp_data')
    print('crawling start')
except:
    print('crawling start')

start_date = datetime.strptime('2022-04-07', '%Y-%m-%d')
end_date = datetime.strptime('2023-03-22', '%Y-%m-%d')
delta = timedelta(days=1)

dates = []
current_date = start_date

while current_date <= end_date:
    dates.append(current_date.strftime('%Y-%m-%d'))
    current_date += delta


data_list = []
for date in dates:
    print(date)
    data_dict = {
        "기준시간": [],
        "공급능력": [],
        "현재부하": [],
        "공급예비력": [],
        "운영예비력": [],
        "운영예비율": [],
    }
    url = f'http://www.happydr.co.kr/?act=&mid=CurrentSupplyView&vid=&view=chart&option_date={date}'

    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    tbody = soup.find('tbody')

    trs = tbody.find_all('tr', {'y': ''})

    for tr in trs:
        row = [td.text for td in tr.find_all('td')]

        tmp = {
            column: crawl_data
            for column, crawl_data in zip(data_dict, row)
        }
        data_list.append(tmp)

    df = pd.DataFrame(data_list)
    df.to_csv(f'./tmp_data/real_sup_electro_{str(date)}.csv')


### 전처리

In [45]:
total_df = pd.read_excel('G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 4\한국전력거래소_5분단위 전력수급현황.xlsx')
total_df

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00:00,64355.0,51726.33,59400,12628.67,24.41439,12628.67,24.41439
1,2012-06-01 00:05:00,64355.0,51342.53,60800,13012.47,25.34442,13012.47,25.34442
2,2012-06-01 00:10:00,64355.0,51583.17,60800,12771.83,24.75969,12771.83,24.75969
3,2012-06-01 00:15:00,64355.0,51631.69,60800,12723.31,24.64245,12723.31,24.64245
4,2012-06-01 00:20:00,64355.0,51252.66,60800,13102.35,25.56423,13102.35,25.56423
...,...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35:00,82791.9,63877.30,69300,18914.70,29.61100,10803.00,16.91210
1035063,2022-04-07 23:40:00,82768.1,63525.30,69300,19242.80,30.29160,10696.40,16.83800
1035064,2022-04-07 23:45:00,82767.9,63506.20,69300,19261.70,30.33040,10501.30,16.53580
1035065,2022-04-07 23:50:00,82762.4,62819.60,69300,19942.80,31.74610,10706.30,17.04290


In [43]:
total_df[total_df['공급예비력(MW)'] != total_df['운영예비력(MW)']]

In [52]:
total_df.drop('최대예측수요(MW)', axis=1, inplace=True)
total_df

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00:00,64355.0,51726.33,12628.67,24.41439,12628.67,24.41439
1,2012-06-01 00:05:00,64355.0,51342.53,13012.47,25.34442,13012.47,25.34442
2,2012-06-01 00:10:00,64355.0,51583.17,12771.83,24.75969,12771.83,24.75969
3,2012-06-01 00:15:00,64355.0,51631.69,12723.31,24.64245,12723.31,24.64245
4,2012-06-01 00:20:00,64355.0,51252.66,13102.35,25.56423,13102.35,25.56423
...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35:00,82791.9,63877.30,18914.70,29.61100,10803.00,16.91210
1035063,2022-04-07 23:40:00,82768.1,63525.30,19242.80,30.29160,10696.40,16.83800
1035064,2022-04-07 23:45:00,82767.9,63506.20,19261.70,30.33040,10501.30,16.53580
1035065,2022-04-07 23:50:00,82762.4,62819.60,19942.80,31.74610,10706.30,17.04290


In [54]:
total_df.columns

Index(['기준일시', '공급능력(MW)', '현재수요(MW)', '공급예비력(MW)', '공급예비율(퍼센트)', '운영예비력(MW)',
       '운영예비율(퍼센트)'],
      dtype='object')

In [6]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035067 entries, 0 to 1035066
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   기준일시        1035067 non-null  datetime64[ns]
 1   공급능력(MW)    1035067 non-null  float64       
 2   현재수요(MW)    1035067 non-null  float64       
 3   최대예측수요(MW)  1035067 non-null  int64         
 4   공급예비력(MW)   1035067 non-null  float64       
 5   공급예비율(퍼센트)  1035067 non-null  float64       
 6   운영예비력(MW)   1035067 non-null  float64       
 7   운영예비율(퍼센트)  1035067 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 63.2 MB


In [ ]:
df = pd.read_csv('G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 4\merged_df3_원본.csv')

In [16]:
df['기준시간'] = pd.to_datetime(df['기준시간'])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100800 entries, 0 to 100799
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   기준시간    100800 non-null  datetime64[ns]
 1   공급능력    100800 non-null  int64         
 2   현재부하    100800 non-null  int64         
 3   공급예비력   100800 non-null  int64         
 4   운영예비력   100800 non-null  int64         
 5   운영예비율   100800 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 4.6+ MB


In [23]:
df = df[df['기준시간'] >= '2022-04-08']
df.reset_index(inplace=True, drop=True)

In [40]:
df['운영예비율(퍼센트)'] = df['운영예비율'].apply(lambda x: float(x.split(' ')[0]))

In [64]:
df['운영예비율(퍼센트)'] = df['운영예비력'] / df['현재부하'] * 100
df['공급예비율(퍼센트)'] = df['공급예비력'] / df['현재부하'] * 100
df['공급능력(MW)'] = df['공급능력'] / 1000
df['현재수요(MW)'] = round((df['현재부하'] / 1000), 2) 
df['공급예비력(MW)'] = round((df['공급예비력'] / 1000), 2)
df['운영예비력(MW)'] = round((df['운영예비력'] / 1000), 2)

In [65]:
new_df = df[['기준시간', '공급능력(MW)', '현재수요(MW)', '공급예비력(MW)', '공급예비율(퍼센트)', '운영예비력(MW)', '운영예비율(퍼센트)']]

In [66]:
new_df.columns = ['기준일시', '공급능력(MW)', '현재수요(MW)', '공급예비력(MW)', '공급예비율(퍼센트)', '운영예비력(MW)',
       '운영예비율(퍼센트)']

In [67]:
new_df

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2022-04-08 00:00:00,82759.1,61323.4,21435.7,34.955172,11446.1,18.665143
1,2022-04-08 00:05:00,82641.8,61504.3,21137.6,34.367678,10842.1,17.628198
2,2022-04-08 00:10:00,82641.9,61019.6,21622.3,35.435008,10733.5,17.590250
3,2022-04-08 00:15:00,82644.2,60801.0,21843.2,35.925725,10828.7,17.810069
4,2022-04-08 00:20:00,82624.2,60591.5,22032.7,36.362691,11103.2,18.324683
...,...,...,...,...,...,...,...
100507,2023-03-22 23:35:00,86286.0,63432.0,22436.3,35.370633,11623.3,18.324032
100508,2023-03-22 23:40:00,86287.9,62935.1,23087.9,36.685252,11587.5,18.411824
100509,2023-03-22 23:45:00,86286.5,62616.6,23670.0,37.801478,11484.2,18.340504
100510,2023-03-22 23:50:00,86274.6,62236.5,24038.0,38.623637,11538.5,18.539764


In [62]:
total_df = pd.concat([total_df, new_df], ignore_index=True)
total_df

,기준일시,공급능력(MW),현재수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00:00,64355.0,51726.33,12628.67,24.414390,12628.67,24.414390
1,2012-06-01 00:05:00,64355.0,51342.53,13012.47,25.344420,13012.47,25.344420
2,2012-06-01 00:10:00,64355.0,51583.17,12771.83,24.759690,12771.83,24.759690
3,2012-06-01 00:15:00,64355.0,51631.69,12723.31,24.642450,12723.31,24.642450
4,2012-06-01 00:20:00,64355.0,51252.66,13102.35,25.564230,13102.35,25.564230
...,...,...,...,...,...,...,...
1135574,2023-03-22 23:35:00,86286.0,63432.00,22436.30,35.370633,11623.30,18.324032
1135575,2023-03-22 23:40:00,86287.9,62935.10,23087.90,36.685252,11587.50,18.411824
1135576,2023-03-22 23:45:00,86286.5,62616.60,23670.00,37.801478,11484.20,18.340504
1135577,2023-03-22 23:50:00,86274.6,62236.50,24038.00,38.623637,11538.50,18.539764


In [74]:
total_df.to_csv('G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 4\Total_date.csv', index=False)
# total_df.to_excel('G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 4\Total_date.xlxs', index=False)
with open('G:\내 드라이브\work_space\Project_Files\BI공모전\분석과제 4\Total_date.pkl', 'wb') as f:
    pickle.dump(total_df, f)

In [57]:
df

,기준시간,공급능력,현재부하,공급예비력,운영예비력,운영예비율,운영예비율(퍼센트),공급예비율(퍼센트),공급능력(MW),현재수요(MW),공급예비력(MW),운영예비력(MW)
0,2022-04-08 00:00:00,82759100,61323400,21435700,11446100,18.7 %,18.665143,34.955172,82759.1,61323.4,21435.7,11446.1
1,2022-04-08 00:05:00,82641800,61504300,21137600,10842100,17.6 %,17.628198,34.367678,82641.8,61504.3,21137.6,10842.1
2,2022-04-08 00:10:00,82641900,61019600,21622300,10733500,17.6 %,17.590250,35.435008,82641.9,61019.6,21622.3,10733.5
3,2022-04-08 00:15:00,82644200,60801000,21843200,10828700,17.8 %,17.810069,35.925725,82644.2,60801.0,21843.2,10828.7
4,2022-04-08 00:20:00,82624200,60591500,22032700,11103200,18.3 %,18.324683,36.362691,82624.2,60591.5,22032.7,11103.2
...,...,...,...,...,...,...,...,...,...,...,...,...
100507,2023-03-22 23:35:00,86286000,63432000,22436300,11623300,18.2 %,18.324032,35.370633,86286.0,63432.0,22436.3,11623.3
100508,2023-03-22 23:40:00,86287900,62935100,23087900,11587500,18.3 %,18.411824,36.685252,86287.9,62935.1,23087.9,11587.5
100509,2023-03-22 23:45:00,86286500,62616600,23670000,11484200,18.3 %,18.340504,37.801478,86286.5,62616.6,23670.0,11484.2
100510,2023-03-22 23:50:00,86274600,62236500,24038000,11538500,18.5 %,18.539764,38.623637,86274.6,62236.5,24038.0,11538.5
